# S&P 2019 Data Wrangling

**by Marc Angelo Acebedo**

## Table of Contents
- [Introduction](#introduction)
- [Data Wrangling](#wrangling)
    - [Gather](#gather)
    - [Assess](#assess)
    - [Clean](#clean)
    - [Store](#store)

## I) Introduction <a id = "introduction">

**Broad question:** How do price forecasts for each firm in the S&P 2019 Index compare to their corresponding actual prices?

**Approach:** Analyze difference in means between average forecast EPS and average actual EPS for each firm.

I will be analyzing quarterly price returns within the past 20 years for the firms present in the S&P 500 2019 Index.

> At first, I wanted to analyze the forecasted vs. actual price earnings of the S&P in its entirety for the past 20 years. However, considering that firms continuously enter and leave stock indices every year, there would be varying levels of inconsistencies and marginal errors when comparing annual S&P returns alone. To combat this problem, I have isolated these two approaches:
- Analyze the historical earnings of *only* the firms present in the S&P 2019 Index
- Keep track of all firms that were present in the S&P for the past 20 years. Keep track of how many times each firm appeared in the Index and for those with the least count, analyze them individually on how they differ from the firms that stayed for longer.

[TK] HEre is a breakdown of my final clean CSV's features, TK.csv.

## II) Data Wrangling <a id="wrangling"></a>

To gather the data depicted under the `./data` folder, I used Bloomberg Excel functions.

### A) Gather <a id = "gather"></a>
> **APPROACH 1:** Focus on the firms that appear in the 2019 S&P Index and analyze their forecasted vs. actual price earnings for the last 20 years.

To ensure consistency in analysis among multiple firms, I divide both the forecasted and actual price earning dates by *calendar period* instead of fiscal period. This is because fiscal period differs by firm whereas calendar period is consistent by dates. 

#### Through the Bloomberg Excel functions, I gathered four datasets with different purposes:

- historical forecasted EPS
- historical actual EPS
- historical actual EOD price
- historical forecasted EPS relying on terms

---
Before delving into the data, let's define what above terms:

**EPS**

> EPS stands for ***Estimated Price Earnings.*** The formal definition of EPS given by Investopedia is this:

Earnings per share is the portion of a company's profit that is allocated to each outstanding share of a common stock, serving as an indicator of the company's financial health.

In other words, the EPS is a portion of the company's **net income** after all of their dividends are paid off. Dividends are profits that are paid out to shareholders of the company. EPS is one of the most useful and valuable financial measurements because they ***determine a stock's worth.*** The higher the stock, the more the company can pay out dividends to its shareholders, and the more net profit they are determined to generate.

$$ EPS = \frac{Net Income - Preferred Dividends}{Weighted Average Common Shares Outstanding}\$$

**EOD**

> EOD stands for the ***End of Day*** price. For any given day, the EOD marks the ***price at which the stock was valued*** at the end of the day's trading period.

In [1]:
PATH = './data/'
PATH_CLEAN = './data/clean/'
PATH_CLEAN_AVGS = './data/clean/averages/'
PATH_CLEAN_AVGS_SUB = './data/clean/averages/components/'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from functools import reduce

### Import Data <a id = "import"></a>

### Let's summarize the contents of the following DataFrames as we import them.
> All DataFrames consist of the 505 firms found in the 2019 S&P Index with EPS and EOD data encompassing 20 years: from January 1999 until the December 2019.

**Historic forecasted EPS**
> According to Investopedia, consensus estimates is normally an average or median of all the forecasts from individual analysts tracking a particular stock. In this case, the consensus estimate is for ***EPS for each firm present in the index as of 2019.*** Forecasted EPS is calculated by ***quarterly earnings,*** usually by each firm's fiscal period. Estimates of quarterly earnings are published at the beginning of each quarterly period.

In [2]:
#historic forecasted EPS 
eps_fc = pd.read_csv(PATH + 'sp-eps-fc.csv')

In [3]:
eps_fc.head()

,Term Forecasted,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,1999Q1,NaN,NaN,NaN,0.025,NaN,NaN,-0.10,0.423,NaN,...,0.456,0.090,0.239,0.123,NaN,NaN,0.123,NaN,0.622,NaN
1,1999Q2,NaN,NaN,NaN,0.020,NaN,NaN,-0.13,0.420,NaN,...,0.213,0.090,0.264,0.135,NaN,NaN,0.150,NaN,0.661,NaN
2,1999Q3,NaN,NaN,NaN,0.023,NaN,NaN,-0.11,0.381,NaN,...,0.767,0.099,0.300,0.129,NaN,NaN,0.171,NaN,0.693,NaN
3,1999Q4,NaN,NaN,NaN,0.016,NaN,NaN,NaN,0.432,NaN,...,0.432,0.124,0.386,0.178,NaN,NaN,0.164,NaN,0.667,NaN
4,00Q1,NaN,NaN,NaN,0.032,NaN,NaN,NaN,0.441,NaN,...,0.283,0.143,0.436,0.138,NaN,NaN,0.138,NaN,0.668,NaN


**Historic actual EPS**
> Unlike forecasted EPS, actual EPS are the real numbers denoting Earnings-per-Share for a singular firm. Historic actual EPS will be compared to forecasted EPS to draw correlations and comparisons.

In [4]:
#historic actual EPS
eps_act = pd.read_csv(PATH + 'sp-eps-act.csv')

In [5]:
eps_act.head()

,Quarter,Year,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,Q1,1999,0.16,0.99,NaN,0.040000,NaN,0.0925,-0.030,0.44,...,0.34,0.0000,0.210,0.123333,1.64,NaN,0.1725,NaN,0.56,NaN
1,Q2,1999,0.35,1.76,NaN,0.035357,NaN,0.1000,-0.145,0.42,...,0.06,0.0975,0.285,0.133333,2.28,NaN,0.2900,NaN,0.60,NaN
2,Q3,1999,0.30,1.86,NaN,0.050357,NaN,0.1075,-0.135,0.30,...,0.63,0.0325,0.315,0.130000,1.96,NaN,0.3200,NaN,0.64,NaN
3,Q4,1999,0.32,1.89,NaN,0.024643,NaN,0.0375,-0.080,0.43,...,0.43,0.1300,0.600,0.180000,1.72,NaN,0.2375,NaN,0.49,NaN
4,Q1,2000,0.29,0.89,NaN,0.040585,NaN,0.1050,-0.045,0.45,...,0.45,0.1650,0.500,0.140000,-1.48,NaN,0.2025,NaN,-0.33,NaN


**Historic actual EOD**
> Though this is not directly related to EPS data, EOD would be an interesting measure to use when generating intriguing visualizations and analyses. Who knows what visuals and conclusions I would arrive to with this measure. 

In [6]:
#historic actual EOD
eod_act = pd.read_csv(PATH + 'sp-eod-act.csv')

In [7]:
eod_act.head()

,date,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,3/31/1999,NaN,NaN,NaN,1.2835,NaN,8.2934,6.250,20.9504,NaN,...,NaN,20.2813,35.2813,7.7500,140.6214,NaN,12.6284,NaN,66.5000,NaN
1,6/30/1999,NaN,NaN,NaN,1.6540,NaN,6.1859,6.875,20.3630,NaN,...,NaN,28.6250,38.5625,9.6250,155.6057,NaN,9.7297,NaN,63.5000,NaN
2,9/30/1999,NaN,NaN,NaN,2.2612,NaN,5.7462,7.750,16.4471,NaN,...,NaN,32.7656,37.9688,7.5833,110.4883,NaN,7.3591,NaN,55.1250,NaN
3,12/31/1999,52.0909,NaN,NaN,3.6719,NaN,3.6843,18.375,16.2513,NaN,...,NaN,45.4688,40.2813,7.8750,59.7723,NaN,6.9434,NaN,59.1875,NaN
4,3/31/2000,70.0721,NaN,NaN,4.8504,NaN,3.6388,20.250,15.7478,NaN,...,NaN,82.8125,38.9063,9.4583,68.4994,NaN,5.5839,NaN,41.6250,NaN


**Historic forecasted EPS 3 months prior**
> Instead of using forecast data collected at the beginning of the fiscal period, this feature contains EPS data projected 3 months before the current fiscal period. This is an interesting metric to see how differently forecasters make their predictions at different times.

In [8]:
#historic forecasted EPS 3-months prior
eps_fc_terms = pd.read_csv(PATH + 'sp-eps-fc-terms.csv')

In [9]:
eps_fc_terms.head()

,Forecast Made,Term Forecasted,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
0,10/1/1999,00Q1,NaN,NaN,NaN,0.030,NaN,NaN,NaN,0.466,...,NaN,0.143,0.398,0.140,NaN,NaN,0.144,NaN,0.777,NaN
1,1/1/2000,00Q2,NaN,NaN,NaN,0.026,NaN,NaN,NaN,0.437,...,NaN,0.168,0.378,0.153,NaN,NaN,0.184,NaN,0.790,NaN
2,4/1/2000,00Q3,NaN,NaN,NaN,0.029,NaN,NaN,NaN,0.414,...,0.84,0.189,0.425,0.149,NaN,NaN,0.218,NaN,0.803,NaN
3,7/1/2000,00Q4,NaN,NaN,NaN,0.032,NaN,NaN,-0.18,0.477,...,0.53,0.211,0.487,0.200,NaN,NaN,0.244,NaN,0.777,NaN
4,9/1/2000,01Q1,NaN,NaN,NaN,0.041,NaN,NaN,-0.09,0.495,...,0.40,0.259,0.475,0.158,NaN,NaN,0.178,NaN,0.750,NaN


### B) Assess <a id = "assess"></a>

> The following DataFrames contain data for each firm across various dates. To account for all firm averages, my goal is to generate a CSV file where each row contains the firm average, with the features as columns.

In [10]:
dict_dfs = {'eps_fcast' : eps_fc,
           'eps_actual' : eps_act,
           'eod_actual' : eod_act,
           'eps_fcast_terms' : eps_fc_terms}

### Check Missing Data

In [11]:
for key, df in dict_dfs.items():
    print(key, df.shape)

eps_fcast (84, 506)
eps_actual (84, 507)
eod_actual (84, 506)
eps_fcast_terms (80, 507)


***We need to make sure the number of firms in each DataFrame is consistent.***

---

**Observation 1:** for `eps_fcast`, there are 505 firms encompassing 84 quarterly fiscal periods since 1999.

> There are 506 columns: 1 column being `Term Forecast`, the rest firm names.

**Observation 2:** for `eps_actual`, there are 505 firms encompassing 84 quarterly fiscal periods since 1999.

> There are 507 columns: 2 columns being `Quarter` and `Year`, the rest firm names.

**Observation 3:** for `eod_actual`, there are 505 firms encompassing 84 quarterly calendar periods since 1999.

> There are 506 columns: 1 column being `date`, the rest firm names.

**Observation 4:** for `eps_fcast_terms`, there are 505 firms encompassing 80 quarterly calendar periods since 1999.

> There are 507 columns: 2 columns being `Forecast Made` and `Term Forecasted`, the rest firm names.
 
Since there are only 80 quarterly calendar periods, that ***implies an entire year is missing.***

**Observation 5:** For `eps_fcast`, `eps_actual`, and `eod_actual`, since there are 4 quarters in a year, 84 quarterly forecast periods equate to 21 years. This is correct since we are analyzing the years from 1999 until the end of 2019.

### Most importantly, the number of firms across all DataFrames is consistent.

----

***Check which year is missing from `eps_fcast_terms`***.

> Each **Term Forecasted** entry under `eps_fcast_terms` records the year with 2 digits, so Quarter 1 of the year 2000 becomes 00Q1.
- isolate the first 2 characters to get the year
- join a '20' in front of the string so 00 becomes 2000
- list the number of unique values.

In [12]:
#iterate over all years in eps_fcast_terms, append '20' in front of string
list(map(lambda x: '20' + x, eps_fc_terms['Term Forecasted'].str[:2].unique()))

['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019']

**Observation**: The year 1999 is missing from `eps_fc_terms`. This makes sense because the start of the forecasting period would be in the last quarter of 1999, which is October. 

---

### Check Nulls

In [13]:
for key, df in dict_dfs.items():
    #display (row, column) per DataFrame
    print('# NaN in {}: {}'.format(key, df.isna().sum().sum()))

# NaN in eps_fcast: 7055
# NaN in eps_actual: 5021
# NaN in eod_actual: 6921
# NaN in eps_fcast_terms: 7080


**Observation:** All four DataSets contain null values.
> In order to combat this, we'll have to look at both the **number of rows** and **number of columns** with missing data, separately. This way, we can isolate which firms and/or time periods contain complete or incomplete data.

In [14]:
#check rows for missing data
for key, df in dict_dfs.items():
    num_rows_missing = df.isna().sum().max()
    print('{} has {} time periods containing missing data, out of {} total rows.'.format(key, num_rows_missing, df.shape[0]))

eps_fcast has 82 time periods containing missing data, out of 84 total rows.
eps_actual has 84 time periods containing missing data, out of 84 total rows.
eod_actual has 83 time periods containing missing data, out of 84 total rows.
eps_fcast_terms has 80 time periods containing missing data, out of 80 total rows.


In [15]:
len(eps_fc.columns[eps_fc.isnull().any()])

254

In [16]:
#check columns for missing data
for key, df in dict_dfs.items():
    cols_missing = df.columns[df.isnull().any()]
    num_cols_missing = len(cols_missing)
    print('{} has {} firms containing missing data, out of {} total columns.'
         .format(key, num_cols_missing, df.shape[1]))

eps_fcast has 254 firms containing missing data, out of 506 total columns.
eps_actual has 432 firms containing missing data, out of 507 total columns.
eod_actual has 505 firms containing missing data, out of 506 total columns.
eps_fcast_terms has 340 firms containing missing data, out of 507 total columns.


**Observation 1:** The only two datasets with incomplete data for all time periods are **actual EPS** and **forecasted EPS 3 months prior.**

> To address this problem, it'd be helpful to isolate the time period ranges for the datasets with incomplete data by row, **forecasted EPS** and **actual EOD price.**

**Observation 2:** For all datasets, all firms contain incomplete data across all time periods.
> This is expected, as analyzing financial history spanning over 20 years will naturally be rife with missing and inaccurate data. The ***good news is that `eps_fcast`, `eps_actual`, and `eps_fcast_terms` are the most complete, while `eod_actual` contains the most amount of missing data.***


**Moving forward, we need to make sure that these inconsistencies won't clash with our analysis.**
> **My approach:** instead of looking at rows and columns ***with*** missing data, we'll be looking at rows and columns that ***are all missing data.***

I figured that if there is some missing data here and there scattered throughout the matrix, then that should not skew our analysis too much.

However, if there a significant amount of rows/columns that are entirely empty, then we ***might have to get ready to drop some dates and firms from our data overall.***

In [17]:
#check for empty rows, return False if row contains at least one non-null value, True if all are null
for key, df in dict_dfs.items():
    cols_check = df.columns
    num_empty_rows = (df[cols_check].isnull().apply(lambda x: all(x), axis = 1)).value_counts()
    print(key, '\n', num_empty_rows, '\n----')

eps_fcast 
 False    84
dtype: int64 
----
eps_actual 
 False    84
dtype: int64 
----
eod_actual 
 False    84
dtype: int64 
----
eps_fcast_terms 
 False    80
dtype: int64 
----


**Observation:** All datasets do not contain empty rows.
> This is good news, since we can rely on the firms' averages per row instead of having to drop or limit time periods.

In [18]:
#check for empty columns, return False if column contains at least one non-null value, True if all are null
for key, df in dict_dfs.items():
    cols_check = df.columns
    num_empty_cols = df[cols_check].isnull().apply(lambda x: all(x), axis = 0).value_counts()
    print(key, '\n', num_empty_cols)

eps_fcast 
 False    506
dtype: int64
eps_actual 
 False    506
True       1
dtype: int64
eod_actual 
 False    506
dtype: int64
eps_fcast_terms 
 False    506
True       1
dtype: int64


**Observation:** `eps_actual` and `eps_fcast_terms` are the only datasets that have an empty column.
> Let's isolate and look at the singular empty column for both DataFrames.

In [19]:
#create function to return an array of column names containing empty data
def comb_cols(df):
    empty_cols = []
    for column in df:
        if df[column].isnull().all():
            empty_cols.append(column)
            
    return empty_cols

In [20]:
#comb datasets for empty columns
print('In eps_act, the firm {} has no data.'.format(comb_cols(eps_act)))
print('In eps_fc_terms, the firm {} has no data.'.format(comb_cols(eps_fc_terms)))

In eps_act, the firm ['AMCR UN Equity'] has no data.
In eps_fc_terms, the firm ['AMCR UN Equity'] has no data.


**Observation:** The same firm in both datasets is empty of data.
> Though this is an annoying error to deal with, it still is to our advantage that both datasets ***share one firm*** in common for missing data. This way, we don't have to worry about dropping two entire firms.

---

### Check Duplicate Data

In [21]:
#check for duplicate data across all rows and columns
for key, df in dict_dfs.items():
    print(key, df.duplicated().sum())

eps_fcast 0
eps_actual 0
eod_actual 0
eps_fcast_terms 0


**Observation:** For all datasets, there are ***no duplicate data.*** This is good news!

**Next, I will check for duplicated firm names.** Although the presence of duplicated firm names will inherently imply duplicated data, sometimes data gets dispersed in weird, unexpected ways, especially when dealing with large datasets.

In [22]:
#check for duplicated firm names
for key, df in dict_dfs.items():
    print(key, df.columns.duplicated().sum())

eps_fcast 0
eps_actual 0
eod_actual 0
eps_fcast_terms 0


**Observation:** For all datasets, there are ***no duplicate firm names.*** This is also good news.

### There is no need to dedupe our data during the cleaning stage.

---

### Check Data Types

> Make sure all numerical data types are consistent.
- EPS and EOD values must all be **floats**
- Dates should all be **objects** (for now)

- `eps_fc` contains ***1 date field,***, so we should expect ***1 object type.***
- `eps_act` contains ***2 date fields,*** so we should expect ***2 object types.***
- `eod_act` contains ***1 date field,*** so we should expect ***1 object type.***
- `eps_fc_terms` contains ***2 date fields,*** so we should expect ***2 object types.***

> And everything else should be a ***float*** type.

**Examine each dataset's overall data types per column.**

In [23]:
for key, df in dict_dfs.items():
    print(key,'\n', df.dtypes.value_counts(), '\n--------')

eps_fcast 
 float64    505
object       1
dtype: int64 
--------
eps_actual 
 float64    499
object       7
int64        1
dtype: int64 
--------
eod_actual 
 float64    505
object       1
dtype: int64 
--------
eps_fcast_terms 
 float64    505
object       2
dtype: int64 
--------


**Observation 1:** `eps_fc`, `eod_act`, and `eps_fc_terms` are all consistent with the expected number of object types.

> But we'll still have to double check these facts later.

**Observation 2:** `eps_act` contains 5 more object columns than expected.

---

***Isolate 'Object' Columns under `eps_act`***

In [24]:
eps_act.select_dtypes(include = 'object').head()

,Quarter,BRK/B UN Equity,FOX UW Equity,GOOG UW Equity,HCP UN Equity,SYMC UW Equity,UA UN Equity
0,Q1,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
1,Q2,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
2,Q3,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
3,Q4,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable
4,Q1,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable,#N/A Field Not Applicable


**Observation:** These firms are all 'Object' types because there is no recorded data under them.
> This is actually missing data. From `eps_act`, there are 6 firms that are empty of data.

### This was a tricky situation to spot, but I caught it in the end. There actually is missing data after all, but we missed those 5 entire firms during the "Check Missing Data" stage, because they were recorded as objects. This means that Pandas wrongly recognized these 5 firms as "complete."

### Quality

**Missing Data**
> **In the end, I decided not to drop any rows or columns with missing data. For the sake of data preservation, I kept all missing data recorded as NaN.**

-  `eps_fc_terms` is missing the year 1999.
- `eps_act` and `eps_fc_act` have one firm with empty data: 'AMCR UN Equity'
- `eps_act` contains 7 empty firms: BRK/B UN Equity, FOX UW Equity, GOOG UW Equity, HCP UN Equity, SYMC UW Equity, UA UN Equity
--- 
- Unnormalized date formats among all DataFrames.
- Firm names across all DataFrames are capitalized and contain white space.
- Erroneous data type for `eps_act` Object columns.
- Erroneous data type for **date** under `eod_act`
- Erroneous data type for **forecast_made** under `eps_fc_terms`
- Erroneous data types for **term_forecast** under `eps_act`, `eps_fc`, and `eps_fc_terms` to DateTime quarter index
- No recorded quarterly data for `eod_act`
---
- Firm names not referenced by **firm_id**
- No recorded 20-year for each dataset.
- No recorded yearly averages for each dataset
- No recorded quarterly averages for each dataset.
---
- Features datasets 

### Tidiness
- Firm 20-year, yearly, and quarterly averages contained in different DataFrames.
- Features data is unnormalized.


### C) CLEAN <a id="clean"></a>

In [25]:
eps_fc_clean = eps_fc.copy()
eps_act_clean = eps_act.copy()
eod_act_clean = eod_act.copy()
eps_fc_terms_clean = eps_fc_terms.copy()

In [26]:
#create dictionary of all clean dfs for iteration
dict_clean = {'eps_fc_clean' : eps_fc_clean,
             'eps_act_clean' : eps_act_clean,
             'eod_act_clean' : eod_act_clean,
             'eps_fc_terms_clean' : eps_fc_terms_clean}

### Code
> **ISSUE 1:**  Unnormalized date formats among all DataFrames.

**Define:** 
- Conjoin `eps_act_clean` dates from 2 columns into 1 to match `eps_fc_clean` format.
> The format we want is YYYYQN. For example, Quarter 1 in 2005 will be 2005Q1.

- Rename both fiscal periods under `eps_act_clean` and `eps_fc_clean` to **term_forecast.**

- For both fiscal periods under `eps_fc_terms_clean`, add an underscore.
- Under `eps_fc_terms_clean`, format **term_forecasted** to YYYYQN.

In [27]:
#create new column, combine year and quarter into 1 string
eps_act_clean['term_forecast'] = eps_act_clean['Year'].map(str) + eps_act_clean['Quarter']

In [28]:
#drop 'Year' and 'Quarter'
eps_act_clean.drop(['Year', 'Quarter'], axis = 1, inplace = True)

In [29]:
#designate new column as first column
reorder_cols = eps_act_clean.columns.tolist()
reorder_cols.insert(0, reorder_cols.pop(reorder_cols.index('term_forecast')))

eps_act_clean = eps_act_clean.reindex(columns = reorder_cols)

In [30]:
#rename eps_fc_clean fiscal period to term_forecast
eps_fc_clean.rename(index = str, columns = {'Term Forecasted' : 'term_forecast'}, inplace = True)

#reassign to dictionary
dict_clean['eps_act_clean'] = eps_act_clean

In [31]:
eps_fc_clean[4:].term_forecast = '20' + eps_fc_clean[4:].term_forecast.astype(str)

D:\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [32]:
#add an underscore for both date periods under eps_fc_terms_clean
eps_fc_terms_clean.rename(index = str, columns = {'Forecast Made' : 'forecast_made',
                                                 'Term Forecasted' : 'term_forecast'},
                         inplace = True)

In [33]:
#under term_forecaste, add a '20' before each string
eps_fc_terms_clean.term_forecast = '20' + eps_fc_terms_clean.term_forecast.astype(str)

### Test

In [34]:
#give overview of both DataFrames
eps_fc_clean.sample(3)

,term_forecast,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
65,2015Q2,0.386,2.597,2.252,2.160,1.061,1.186,0.020,0.503,1.072,...,0.413,0.559,1.113,0.683,NaN,0.430,0.629,1.566,NaN,0.38
81,2019Q2,0.716,1.795,2.207,2.368,2.204,1.955,0.752,0.803,1.573,...,0.517,0.880,0.659,0.631,0.882,0.791,0.873,1.913,1.097,0.82
41,2009Q2,0.127,-1.297,0.835,0.160,NaN,0.449,-0.216,0.887,0.614,...,0.248,0.381,0.993,0.480,NaN,NaN,0.435,0.963,NaN,NaN


In [35]:
eps_act_clean.sample(3)

,term_forecast,A UN Equity,AAL UW Equity,AAP UN Equity,AAPL UW Equity,ABBV UN Equity,ABC UN Equity,ABMD UW Equity,ABT UN Equity,ACN UN Equity,...,XEL UW Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,XYL UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,ZTS UN Equity
19,2003Q4,0.03,-0.70,0.28,0.008571,NaN,0.2675,NaN,0.60,0.26,...,1.20,0.15,1.01,0.32,1.00,NaN,0.365,0.15,1.06,NaN
78,2018Q3,0.74,0.81,1.57,2.360000,1.81,1.2600,0.30,0.32,1.63,...,0.96,-0.05,1.46,0.13,0.34,0.73,1.430,0.80,1.11,0.72
64,2015Q1,0.19,1.34,2.02,3.080000,0.64,-0.9100,-0.04,1.52,1.32,...,0.30,0.65,1.17,0.46,0.80,0.35,0.830,1.01,0.37,0.33


In [36]:
#check consistent format between both `term_forecast` fields
forecasts = pd.concat([eps_fc_clean.term_forecast, eps_act_clean.term_forecast])

#count number of instances where unique term_forecast values are the same
(forecasts.value_counts().sort_index() == 2).value_counts()

True    84
Name: term_forecast, dtype: int64

In [37]:
#check that dates are the first columns
for key, df in dict_clean.items():
    print(df.columns[:2])

Index(['term_forecast', 'A UN Equity'], dtype='object')
Index(['term_forecast', 'A UN Equity'], dtype='object')
Index(['date', 'A UN Equity'], dtype='object')
Index(['forecast_made', 'term_forecast'], dtype='object')


## The **term_forecast** fields under `eps_fc_clean`, `eps_act_clean`, and `eps_fc_terms_clean` are all normalized to YYYQN format. This means `eod_act_clean` is the only one with a different date formatting rule.

### Code
> **ISSUE 2:** Firm names across all DataFrames are capitalized and contain white space.

**Define:** Iterate across all DataFrames. Get rid of everything from the first whitespace character onward. Lowercase column names.

In [38]:
#lowercase names
for key, df in dict_clean.items():
    lower_cols = [x.lower() for x in df.columns]
    df.columns = lower_cols

In [39]:
#get rid of white space
for key, df in dict_clean.items():
    stock_symbols = [x.split(' ', 1)[0] for x in df.columns]
    df.columns = stock_symbols

### Test

In [40]:
#check that all firm names are consistent, should return 505 firm results
all_ticks = []
for key, df in dict_clean.items():
    firm_ticks = np.array(df.columns.values)
    all_ticks.append(firm_ticks)
    
#flatten array
all_ticks = np.hstack(all_ticks)

#assign tick names to Pandas Series
all_ticks = pd.Series(all_ticks)

In [41]:
#look at lowest 3 value counts, SHOULD BE DATE PERIODS
all_ticks.value_counts().sort_values()[:5]

date             1
forecast_made    1
term_forecast    3
udr              4
pvh              4
dtype: int64

In [42]:
#look at highest value counts, should be FOUR (because there are 4 dataframes all with firm names)
all_ticks.value_counts().sort_values(ascending = False)[:5]

hca     4
sbux    4
vrtx    4
unm     4
var     4
dtype: int64

In [43]:
#check for any upper case instance among columns
all_ticks.str.isupper().any()

False

In [44]:
#check for any white space
all_ticks.str.contains(' ').any()

False

### Code
> **ISSUE 3:** Erroneous data type for `eps_act` Object columns.

**Define:**

- Convert '#N/A Field Not Applicable' strings into 'NaN'
- Convert the 6 `eps_act_clean` Object columns into NaN type.

In [45]:
#isolate 6 firm names recorded as 'object'
obj_ticks = eps_act_clean.select_dtypes(include = 'object').columns.values

#exclude 'term_forecast'
obj_ticks = obj_ticks[1:]

In [46]:
#convert to NaN
str_replace = '#N/A Field Not Applicable'
eps_act_clean[obj_ticks] = eps_act_clean[obj_ticks].replace(str_replace, np.nan)

### Test

In [47]:
eps_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 506 entries, term_forecast to zts
dtypes: float64(505), object(1)
memory usage: 332.1+ KB


In [48]:
#make sure 'term_forecast' is the only object-type column
eps_act_clean.select_dtypes(include = 'object').columns.values.tolist()

['term_forecast']

### Code

> **ISSUE 4:**  Erroneous data type for **date** under `eod_act`

**Define:** Convert column `date` to DateTime object.

In [49]:
eod_act_clean.date = pd.to_datetime(eod_act_clean.date, infer_datetime_format=True)

### Test

In [50]:
eod_act_clean.sample(2)

,date,a,aal,aap,aapl,abbv,abc,abmd,abt,acn,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
16,2003-03-31,8.8601,NaN,15.417,1.0100,NaN,12.7357,3.90,16.8319,15.5,...,NaN,23.41,34.95,17.395,22.9210,NaN,8.7473,48.63,42.78,NaN
55,2012-12-31,29.2757,NaN,72.350,76.1471,34.16,43.1800,13.46,31.3400,66.5,...,NaN,35.90,86.55,39.610,17.9679,27.1,47.7452,66.66,21.40,NaN


In [51]:
#confirm datetime64 present
eod_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 506 entries, date to zts
dtypes: datetime64[ns](1), float64(505)
memory usage: 332.1 KB


### Code
> **ISSUE 5:** Erroneous data type for **forecast_made** under `eps_fc_terms`.

**Define:**

- Convert column **forecast_made** to DateTime object.

In [52]:
eps_fc_terms_clean.forecast_made = pd.to_datetime(eps_fc_terms_clean.forecast_made, infer_datetime_format=True)

### Test

In [53]:
#confirm presence of datetime64 object
eps_fc_terms_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 0 to 79
Columns: 507 entries, forecast_made to zts
dtypes: datetime64[ns](1), float64(505), object(1)
memory usage: 317.5+ KB


In [54]:
eps_fc_terms_clean.sample(2)

,forecast_made,term_forecast,a,aal,aap,aapl,abbv,abc,abmd,abt,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
66,2016-04-01,2016Q3,0.471,1.937,2.246,1.778,1.303,1.438,0.152,0.580,...,0.828,0.493,0.684,0.696,1.128,0.53,1.095,1.826,0.491,0.467
33,2008-01-01,2008Q2,0.519,1.275,0.727,0.155,NaN,0.396,-0.210,0.796,...,NaN,0.286,1.972,0.493,1.191,NaN,0.421,1.058,1.288,NaN


### Code
> **ISSUE 6:** Erroneous data types for **term_forecast** under `eps_act`, `eps_fc`, and `eps_fc_terms` to DateTime quarter index

**Define:** Convert YYYYQQ formats into DateTime quarter index type

In [55]:
#helper function to turn a column into Quarter Period type
def convert_period(df, col):
    return pd.to_datetime(df[col]).dt.to_period('Q')
# eps_fc_terms_clean.term_forecast = pd.PeriodIndex(eps_fc_terms_clean.term_forecast, freq='Q')

In [56]:
#convert term_forecast fields to Period type
eps_fc_clean.term_forecast = convert_period(eps_fc_clean, 'term_forecast')
eps_act_clean.term_forecast = convert_period(eps_act_clean, 'term_forecast')
eps_fc_terms_clean.term_forecast = convert_period(eps_fc_terms_clean, 'term_forecast')

### Test

In [57]:
#verify Period Object types
print(eps_fc_clean.dtypes['term_forecast'])
print(eps_act_clean.dtypes['term_forecast'])
print(eps_fc_terms_clean.dtypes['term_forecast'])

period[Q-DEC]
period[Q-DEC]
period[Q-DEC]


### Code
> **ISSUE 7:** No recorded quarterly data for `eod_act`

**Define:** Add a new column **term_forecast** denoting *year* and *quarter*

In [58]:
eod_act_clean['term_forecast'] = eod_act_clean.date.dt.to_period('Q')

In [59]:
#reorder columns
eod_cols = ['date', 'term_forecast'] + [col for col in eod_act_clean.iloc[:, 1:] if col!='term_forecast']

In [60]:
eod_act_clean = eod_act_clean.reindex(columns = eod_cols)

### Test

In [61]:
#verify column order
eod_act_clean.head()

,date,term_forecast,a,aal,aap,aapl,abbv,abc,abmd,abt,...,xel,xlnx,xom,xray,xrx,xyl,yum,zbh,zion,zts
0,1999-03-31,1999Q1,NaN,NaN,NaN,1.2835,NaN,8.2934,6.250,20.9504,...,NaN,20.2813,35.2813,7.7500,140.6214,NaN,12.6284,NaN,66.5000,NaN
1,1999-06-30,1999Q2,NaN,NaN,NaN,1.6540,NaN,6.1859,6.875,20.3630,...,NaN,28.6250,38.5625,9.6250,155.6057,NaN,9.7297,NaN,63.5000,NaN
2,1999-09-30,1999Q3,NaN,NaN,NaN,2.2612,NaN,5.7462,7.750,16.4471,...,NaN,32.7656,37.9688,7.5833,110.4883,NaN,7.3591,NaN,55.1250,NaN
3,1999-12-31,1999Q4,52.0909,NaN,NaN,3.6719,NaN,3.6843,18.375,16.2513,...,NaN,45.4688,40.2813,7.8750,59.7723,NaN,6.9434,NaN,59.1875,NaN
4,2000-03-31,2000Q1,70.0721,NaN,NaN,4.8504,NaN,3.6388,20.250,15.7478,...,NaN,82.8125,38.9063,9.4583,68.4994,NaN,5.5839,NaN,41.6250,NaN


In [62]:
#verify 84 columns
eod_act_clean.shape[0]

84

In [63]:
eod_act_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Columns: 507 entries, date to zts
dtypes: datetime64[ns](1), float64(505), period[Q-DEC](1)
memory usage: 332.8 KB


## All full years are normalized to YYYY-MM-DD, and all quarterly records to YYYY-QQ.

## This is to enable for more efficient handling, cleaning, and classifying of data later on.

### Code

**ISSUE 7:** Firm names not referenced by **firm_id**

**Define:**
- Assign a **firm_id** to each firm for future normalization.

- Generate new CSV named `firms.csv`

In [129]:
#get firm names
firm_names = eps_act_clean.columns[1:].to_numpy()

In [149]:
#assign ids to firm names
firm_names_ids = {}
id = 0

for firm in firm_names:
    firm_names_ids.update({id: firm})
    id += 1

In [153]:
#generate new DF
firm_ids = pd.DataFrame(list(firm_names_ids.items())).rename(columns = {0: 'firm_id', 1: 'firm'})

### Code
> **ISSUE 7:** No recorded 20-year averages for each dataset.

**Define:**


- Isolate 20-year averages for each firm into its own DataFrame

- Create new DataFrame `twenty_avgs` depicting all 20-year averages for each firm

In [74]:
#grab all firm names
firm_names = eps_act_clean.columns.values[1:]

In [118]:
def get_twenty_yr_avg(df, index_names, col_name):
    return pd.DataFrame({col_name : df[index_names].mean()}).rename_axis('firm').reset_index()

In [119]:
#get 20-year averages
eps_fc_twenty_avg = get_twenty_yr_avg(eps_fc_clean, firm_names, 'eps_fc')
eps_act_twenty_avg = get_twenty_yr_avg(eps_act_clean, firm_names, 'eps_act')
eod_act_twenty_avg = get_twenty_yr_avg(eod_act_clean, firm_names, 'eod_act')
eps_fc_terms_twenty_avg = get_twenty_yr_avg(eps_fc_terms_clean, firm_names, 'eps_fc_terms')

In [120]:
#put averages into a list
twenty_avg_list = [eps_fc_twenty_avg,
                  eps_act_twenty_avg,
                  eod_act_twenty_avg,
                  eps_fc_terms_twenty_avg]

In [121]:
#merge all Series
df_twenty_year_avgs = pd.concat(twenty_avg_list, axis = 1, sort=False)

### Test

In [122]:
df_twenty_year_avgs.sample(5)

,firm,eps_fc,firm,eps_act,firm,eod_act,firm,eps_fc_terms
23,ajg,0.481964,ajg,0.434585,ajg,36.497470,ajg,0.527241
379,pnw,0.832667,pnw,0.791807,pnw,50.813554,pnw,0.895974
433,tap,0.765619,tap,0.746574,tap,49.083602,tap,0.840012
486,wltw,1.686387,wltw,1.092112,wltw,149.069333,wltw,1.887718
146,dov,0.785762,dov,0.837470,dov,41.392570,dov,0.900208


In [70]:
#check missing data
df_twenty_year_avgs.isna().any()

firm
eps_fc          False
eps_act          True
eod_act         False
eps_fc_terms     True
dtype: bool

In [71]:
#check duplicate data
df_twenty_year_avgs.duplicated().any()

False

### Code
> **ISSUE 8:** No recorded yearly averages for each dataset.

**Define:**
- Create 4 separate DataFrames for all attributes
- Rename columns to "Feature_Year" (e.g. eps_fc_1999, eod_act_2000, etc.)
- Outer merge all DataFrames to create new DataFrame, `df_yearly_avgs`, on **firms** attribute

In [72]:
#helper function to calculate yearly average of each DataFrame
def create_df_yearly_avgs(df, col):
    #extract year from dates
    return df.groupby(df[col].dt.year).mean().transpose().rename_axis('firm', axis = 'columns')

In [73]:
#generate DataFrames of all yearly averages
eps_fc_yearly_avgs = create_df_yearly_avgs(eps_fc_clean, 'term_forecast')
eps_act_yearly_avgs = create_df_yearly_avgs(eps_act_clean, 'term_forecast')

#for eod_act, doesn't matter if we use date or term_forecast field
eod_act_yearly_avgs = create_df_yearly_avgs(eod_act_clean, 'date')
eps_fc_terms_yearly_avgs = create_df_yearly_avgs(eps_fc_terms_clean, 'term_forecast')

In [183]:
#store yearly averages
eps_fc_yearly_avgs.to_csv(PATH_CLEAN_AVGS_SUB + 'yearly-avgs-eps-fc.csv', encoding = 'utf-8', index = False)
eps_act_yearly_avgs.to_csv(PATH_CLEAN_AVGS_SUB + 'yearly-avgs-eps-act.csv', encoding = 'utf-8', index = False)
eod_act_yearly_avgs.to_csv(PATH_CLEAN_AVGS_SUB + 'yearly-avgs-eod-act.csv', encoding='utf-8', index = False)
eps_fc_terms_yearly_avgs.to_csv(PATH_CLEAN_AVGS_SUB + 'yearly-avgs-eps-fc-terms.csv', encoding='utf-8', index = False)

In [184]:
#rename yearly average columns
eps_fc_yearly_avgs = eps_fc_yearly_avgs.add_suffix('_eps_fc')
eps_act_yearly_avgs = eps_act_yearly_avgs.add_suffix('_eps_act')
eod_act_yearly_avgs = eod_act_yearly_avgs.add_suffix('_eod_act')
eps_fc_terms_yearly_avgs = eps_fc_terms_yearly_avgs.add_suffix('_eps_fc_terms')

In [185]:
#put DataFrames in array
yearly_avgs_arr = [eps_fc_yearly_avgs,
                  eps_act_yearly_avgs,
                  eod_act_yearly_avgs,
                  eps_fc_terms_yearly_avgs]

In [186]:
#merge all yearly average DataFrames
df_yearly_avgs = pd.concat(yearly_avgs_arr, join='outer', axis=1).fillna(method='ffill')

### Test

In [205]:
df_yearly_avgs.head()

firm,1999_eps_fc,2000_eps_fc,2001_eps_fc,2002_eps_fc,2003_eps_fc,2004_eps_fc,2005_eps_fc,2006_eps_fc,2007_eps_fc,2008_eps_fc,...,2010_eps_fc_terms,2011_eps_fc_terms,2012_eps_fc_terms,2013_eps_fc_terms,2014_eps_fc_terms,2015_eps_fc_terms,2016_eps_fc_terms,2017_eps_fc_terms,2018_eps_fc_terms,2019_eps_fc_terms
a,NaN,NaN,NaN,NaN,NaN,NaN,0.277667,0.39525,0.42375,0.48975,...,0.39525,0.64175,0.81375,0.75850,0.81500,0.51950,0.47700,0.53900,0.65025,0.75975
aal,NaN,NaN,NaN,NaN,NaN,NaN,-1.580000,0.20700,0.38225,-1.21275,...,-0.13525,-0.30800,-0.25750,0.46000,0.98575,2.19850,1.57475,1.20500,1.36500,1.49575
aap,NaN,NaN,0.02850,0.21075,0.3185,0.414,0.520000,0.54375,0.58150,0.61275,...,0.83725,1.16675,1.41650,1.46000,1.76325,2.15025,2.23200,2.00175,1.62300,2.04675
aapl,0.021,0.0285,-0.00575,0.00575,0.0025,0.010,0.041500,0.07175,0.10675,0.17775,...,0.36025,0.74675,1.31125,1.61825,1.52150,1.96425,2.23850,2.20825,2.89700,3.16375
abbv,0.021,0.0285,-0.00575,0.00575,0.0025,0.010,0.041500,0.07175,0.10675,0.17775,...,0.36025,0.74675,1.31125,0.81300,0.78000,1.04450,1.21475,1.36825,1.73575,2.17450


In [187]:
#verify that all columns are, in fact, years
print(eps_fc_yearly_avgs.columns.str[:4], '\n',
      eps_act_yearly_avgs.columns.str[:4], '\n',
      eod_act_yearly_avgs.columns.str[:4], '\n',
      eps_fc_terms_yearly_avgs.columns.str[:4])

Index(['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019'],
      dtype='object', name='firm') 
 Index(['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019'],
      dtype='object', name='firm') 
 Index(['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019'],
      dtype='object', name='firm') 
 Index(['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019'],
      dtype='object', name='firm')


In [188]:
#verify there are 505 firms in each dataframe
print(eps_fc_yearly_avgs.shape[0], '\n',
      eps_act_yearly_avgs.shape[0], '\n',
      eod_act_yearly_avgs.shape[0], '\n',
      eps_fc_terms_yearly_avgs.shape[0])

505 
 505 
 505 
 505


In [189]:
#verify all columns under yearly_avgs contain all fields
att_names = ['eps_fc', 'eps_act', 'eod_act', 'eps_fc_terms']

In [190]:
#split YYYY_eps_fc after first _ occurrence, get only unique values
cols_substrs = df_yearly_avgs.columns.str.split('_', 1).str[1].unique()

In [191]:
all(att_names == cols_substrs)

True

### Code

> **ISSUE 9:** No recorded quarterly averages for each dataset.

**Define:** 

- Parse `eod_act_clean` dates  by **calendar quarter average** with quarterly data in a new DataFrame.
- Create separate DataFrames containing quarterly averages for `eps_fc_clean`, `eps_act_clean`, and `eps_fc_terms_clean` 
- Rename columns to "Quarter_Year_Feature" (e.g. **q1_eps_fc,** etc.)
- Outer merge all DataFrames into a new df `df_quarterly_avgs` on **firms**

In [192]:
#helper function to calculate yearly average of each DataFrame
def create_df_quarterly_avgs(df, col):
    #extract quarter from Period objects
    df =  df.groupby(df[col].dt.strftime('Q%q')).mean().transpose().rename_axis('firm', axis = 'columns')
    
    #lowercase column names
    df.columns = df.columns.str.lower()
    return df

In [193]:
#generate all quarterly average DataFrames
eps_fc_quarter_avgs = create_df_quarterly_avgs(eps_fc_clean, 'term_forecast')
eps_act_quarter_avgs = create_df_quarterly_avgs(eps_act_clean, 'term_forecast')
eod_act_quarter_avgs = create_df_quarterly_avgs(eod_act_clean, 'term_forecast')
eps_fc_terms_quarter_avgs = create_df_quarterly_avgs(eps_fc_terms_clean, 'term_forecast')

In [194]:
#store quarterly averages
eps_fc_quarter_avgs.to_csv(PATH_CLEAN_AVGS_SUB + 'quarter-avgs-eps-fc.csv', encoding = 'utf-8', index = False)
eps_act_quarter_avgs.to_csv(PATH_CLEAN_AVGS_SUB + 'quarter-avgs-eps-act.csv', encoding = 'utf-8', index = False)
eod_act_quarter_avgs.to_csv(PATH_CLEAN_AVGS_SUB + 'quarter-avgs-eod-act.csv', encoding='utf-8', index = False)
eps_fc_terms_quarter_avgs.to_csv(PATH_CLEAN_AVGS_SUB + 'quarter-avgs-eps-fc-terms.csv', encoding='utf-8', index = False)

In [195]:
#rename quarterly average columns
eps_fc_quarter_avgs = eps_fc_quarter_avgs.add_suffix('_eps_fc')
eps_act_quarter_avgs = eps_act_quarter_avgs.add_suffix('_eps_act')
eod_act_quarter_avgs = eod_act_quarter_avgs.add_suffix('_eod_act')
eps_fc_terms_quarter_avgs = eps_fc_terms_quarter_avgs.add_suffix('_eps_fc_terms')

In [196]:
#put DataFrames in array
quarterly_avgs_arr = [eps_fc_quarter_avgs,
                  eps_act_quarter_avgs,
                  eod_act_quarter_avgs,
                  eps_fc_terms_quarter_avgs]

In [197]:
#merge all quarterly average DataFrames
df_quarter_avgs = pd.concat(quarterly_avgs_arr, join='outer', axis=1).fillna(method='ffill')

### Test

In [198]:
df_quarter_avgs.head()

firm,q1_eps_fc,q2_eps_fc,q3_eps_fc,q4_eps_fc,q1_eps_act,q2_eps_act,q3_eps_act,q4_eps_act,q1_eod_act,q2_eod_act,q3_eod_act,q4_eod_act,q1_eps_fc_terms,q2_eps_fc_terms,q3_eps_fc_terms,q4_eps_fc_terms
a,0.490500,0.489200,0.526800,0.617467,0.505714,0.287619,0.191000,0.416667,34.058320,32.998630,31.991835,32.538085,0.542500,0.537857,0.558000,0.627667
aal,-0.187429,0.812071,0.666867,0.064333,-0.869524,0.092381,0.071912,-0.871865,42.735000,38.682500,37.785000,42.010000,0.048769,1.261429,1.159643,0.321929
aap,1.270947,1.213500,1.082833,0.650737,1.194074,1.075614,1.007222,0.670000,77.747000,76.724222,78.780333,74.174778,1.453235,1.262778,1.177556,0.776000
aapl,1.160476,0.820048,0.701381,0.810667,1.259246,0.890255,0.750119,0.878384,53.321171,54.858752,60.959586,51.256220,1.231100,0.912950,0.750250,0.840150
abbv,1.207429,1.340571,1.361571,1.368857,0.963750,0.867500,0.968750,0.249505,64.034286,66.394286,68.447143,66.167143,1.225000,1.293143,1.359429,1.398857


In [199]:
#verify that all columns are, in fact, years
print(eps_fc_quarter_avgs.columns.str[:2], '\n',
      eps_act_quarter_avgs.columns.str[:2], '\n',
      eod_act_quarter_avgs.columns.str[:2], '\n',
      eps_fc_terms_quarter_avgs.columns.str[:2])

Index(['q1', 'q2', 'q3', 'q4'], dtype='object', name='firm') 
 Index(['q1', 'q2', 'q3', 'q4'], dtype='object', name='firm') 
 Index(['q1', 'q2', 'q3', 'q4'], dtype='object', name='firm') 
 Index(['q1', 'q2', 'q3', 'q4'], dtype='object', name='firm')


In [200]:
#verify there are 505 firms in each dataframe
print(eps_fc_quarter_avgs.shape[0], '\n',
      eps_act_quarter_avgs.shape[0], '\n',
      eod_act_quarter_avgs.shape[0], '\n',
      eps_fc_terms_quarter_avgs.shape[0])

505 
 505 
 505 
 505


In [ ]:
#verify all columns under yearly_avgs contain all fields
att_names = ['q1', 'q2', 'q3', 'q4']

In [ ]:
#split YYYY_eps_fc after first _ occurrence, get only unique values
cols_substrs = df_quarter_avgs.columns.str.split('_', 1).str[1].unique()

In [203]:
all(att_names == cols_substrs)

True

### Code
>**ISSUE 10:** Firm 20-year, yearly, and quarterly averages contained in different DataFrames.

**Define:** 
- Outer merge `df_twenty_year_avgs`, `df_yearly_avgs`, and `df_quarter_avgs` on **firm** column.

In [206]:
#put all average DFs in an array
avgs_array = [df_twenty_year_avgs, df_yearly_avgs, df_quarter_avgs]

In [207]:
df_all_avgs = pd.concat(avgs_array, join='outer', axis=1).fillna(method='ffill')

### Test

In [209]:
df_all_avgs.head()

firm,eps_fc,eps_act,eod_act,eps_fc_terms,1999_eps_fc,2000_eps_fc,2001_eps_fc,2002_eps_fc,2003_eps_fc,2004_eps_fc,...,q3_eps_act,q4_eps_act,q1_eod_act,q2_eod_act,q3_eod_act,q4_eod_act,q1_eps_fc_terms,q2_eps_fc_terms,q3_eps_fc_terms,q4_eps_fc_terms
a,0.531678,0.352169,32.896718,0.567579,NaN,NaN,NaN,NaN,NaN,NaN,...,0.191000,0.416667,34.058320,32.998630,31.991835,32.538085,0.542500,0.537857,0.558000,0.627667
aal,0.339879,-0.388520,40.303125,0.709745,NaN,NaN,NaN,NaN,NaN,NaN,...,0.071912,-0.871865,42.735000,38.682500,37.785000,42.010000,0.048769,1.261429,1.159643,0.321929
aap,1.051973,0.996901,76.856583,1.163366,NaN,NaN,0.02850,0.21075,0.3185,0.414,...,1.007222,0.670000,77.747000,76.724222,78.780333,74.174778,1.453235,1.262778,1.177556,0.776000
aapl,0.873143,0.944501,55.145230,0.933613,0.021,0.0285,-0.00575,0.00575,0.0025,0.010,...,0.750119,0.878384,53.321171,54.858752,60.959586,51.256220,1.231100,0.912950,0.750250,0.840150
abbv,1.319607,0.778921,66.260714,1.322593,0.021,0.0285,-0.00575,0.00575,0.0025,0.010,...,0.968750,0.249505,64.034286,66.394286,68.447143,66.167143,1.225000,1.293143,1.359429,1.398857


In [210]:
#verify 103 columns
df_all_avgs.shape

(505, 103)

# III) Store Data <a id = "store"></a>

**Disclaimer:** I decided to put the following CSVs under the ***./data/clean/averages/components*** directory because they form *part* of the main CSVs under the ***./data/clean/averages*** directory:
- `quarter-avgs-eod-act.csv`
- `quarter-avgs-eps-act.csv`
- `quarter-avgs-eps-fc.csv`
- `quarter-avgs-eps-fc-terms.csv`

> The above are all components of the `quarter-avgs.csv` under **/averages/**

- `yearly-avgs-eod-act.csv`
- `yearly-avgs-eps-act.csv`
- `yearly-avgs-eps-fc.csv`
- `yearly-avgs-eps-fc-terms.csv`

> The above are all components of the `yearly-avgs.csv` under **/averages/**

**Additionally, I decided to keep `avgs.csv` under the *./data/clean/averages/* directory.** Although this CSV containing all averages would be highly inefficient for the data exploration stage, I decided it would be best to keep it on hand for convenience.

Using `avgs.csv` would mean having the parse through the column names to isolate 20-year, yearly, and quarterly averages, but all of that redundance can be avoided by simply focusing on the `twenty-year-avgs`, `yearly-avgs`, and `quarter-avgs` CSVs alone.

## Firms

In [156]:
firm_ids.to_csv(PATH_CLEAN + 'firms.csv', encoding = 'utf-8', index = False)

## Averages

In [87]:
#twenty-year-averages
df_twenty_year_avgs.to_csv(PATH_CLEAN_AVGS + 'twenty-year-avgs.csv', encoding = 'utf-8', index = False)

In [215]:
#yearly averagas
df_yearly_avgs.to_csv(PATH_CLEAN_AVGS + 'yearly-avgs.csv', encoding = 'utf-8', index = False)

In [216]:
#quarterly averages
df_quarter_avgs.to_csv(PATH_CLEAN_AVGS + 'quarter-avgs.csv', encoding = 'utf-8', index = False)

In [217]:
#all averages
df_all_avgs.to_csv(PATH_CLEAN_AVGS + 'avgs.csv', encoding = 'utf-8', index = False)

## Clean Standalone Datasets

In [218]:
eps_fc_clean.to_csv(PATH_CLEAN + 'eps-fc.csv')
eps_act_clean.to_csv(PATH_CLEAN + 'eps-act.csv')
eod_act_clean.to_csv(PATH_CLEAN + 'eod-act.csv')
eps_fc_terms_clean.to_csv(PATH_CLEAN + 'eps-fc-terms.csv')

# IV) Notes



- When dealing with the CSVs depicting averages, we need to keep in mind **missing data** in our interpretations.
- I decided to put the years as columns under yearly average DF generation because: the amount of columns generated here, TK, would be less than the amount of 
	firms there are, 505. [TK link to a study showing how having too many columns take up memory vs rows]
    
- keep in mind that `eod_act` is based on **calendar years** not fiscal years.

In [136]:

#convert notebook to HTML